In [1]:
# Import packages
import json
from pathlib import Path

import folium
import matplotlib
import numpy as np
import pandas as pd
from exif import Image

In [2]:
# Create function to turn binary into image
def read_exif_data(file_path: Path) -> Image:
    """Read metadata from photo."""
    with open(file_path, 'rb') as f:
        return Image(f)
    

In [17]:
# Designate image folder
BASE_LOC = Path(r"C:\Users\drift\Desktop\PekelbakProject\Pekelbakken")
file = Path("18.jpg")
img = read_exif_data(BASE_LOC / file)
print('\n'.join([i for i in img.list_all() if i.startswith('gps_')]))


In [4]:
# Function to convert coordinates to decimals
def convert_coords_to_decimal(coords: tuple[float,...], ref: str) -> float:
    """Covert a tuple of coordinates in the format (degrees, minutes, seconds)
    and a reference to a decimal representation.
    Args:
        coords (tuple[float,...]): A tuple of degrees, minutes and seconds
        ref (str): Hemisphere reference of "N", "S", "E" or "W".
    Returns:
        float: A signed float of decimal representation of the coordinate.
    """
    if ref.upper() in ['W', 'S']:
        mul = -1
    elif ref.upper() in ['E', 'N']:
        mul = 1
    else:
        print("Incorrect hemisphere reference. "
              "Expecting one of 'N', 'S', 'E' or 'W', "
              f'got {ref} instead.')
        
    return mul * (coords[0] + coords[1] / 60 + coords[2] / 3600)  

In [5]:
# Function to extract decimal coordinates from image
def get_decimal_coord_from_exif(exif_data: Image) -> tuple[float, ...]:
    """Get coordinate data from exif and convert to a tuple of 
    decimal latitude, longitude and altitude.
    Args:
        exif_data (Image): exif Image object
    Returns:
        tuple[float, ...]: A tuple of decimal coordinates (lat, lon, alt)
    """
    try:
        lat = convert_coords_to_decimal(
            exif_data['gps_latitude'], 
            exif_data['gps_latitude_ref']
            )
        lon = convert_coords_to_decimal(
            exif_data['gps_longitude'], 
            exif_data['gps_longitude_ref']
            )
        alt = exif_data['gps_altitude']
        return (lat, lon, alt)
    except (AttributeError, KeyError):
        print('Image does not contain spatial data or data is invalid.')  
        raise

In [6]:
# Function to read all spatial data from all iamges in folder and store in dictionary
def read_spatial_data_from_folder(folder: Path, image_extension: str = '*.jpg') -> dict[str, dict]:
    """Create a dictionary of spatial data from photos in a folder.
    Args:
        folder (Path): folder as a Path object
        image_extension (str): extension of images to read. 
            Defaults to '.jpg'.
    Returns:
        dict[str, dict]: A dictionary with filename as the key
            and a value of a dictionary if the format:
                {
                    'coordinates': tuple[float, ...],
                    'timestamp': str
                }
    """
    coord_dict = dict()
    source_files = [f for f in folder.rglob(image_extension)]
    exif = [read_exif_data(f) for f in source_files]
    
    for f, data in zip(source_files, exif):
        try:
            coord = get_decimal_coord_from_exif(data)
        except (AttributeError, KeyError):
            continue
        else:
            coord_dict[str(f)] = dict()
            coord_dict[str(f)]['latitude'] = coord[0]
            coord_dict[str(f)]['longitude'] = coord[1]
            coord_dict[str(f)]['altitude'] = coord[2]
        # Also read date when photo was taken (if available)
        try:
            coord_dict[str(f)]['timestamp'] = data.datetime
        except (AttributeError, KeyError):
            print(f"Photo {f.name} does not contain datetime information.")
            coord_dict[str(f)]['timestamp'] = None
    
    return coord_dict

In [7]:
res = read_spatial_data_from_folder(BASE_LOC)


Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.


In [8]:
print(json.dumps(res, indent=4))

{
    "C:\\Users\\drift\\Desktop\\PekkelbakProject\\101.jpg": {
        "latitude": 52.157972694444446,
        "longitude": 4.491737333333333,
        "altitude": 43.0,
        "timestamp": "2018:08:24 18:15:35"
    },
    "C:\\Users\\drift\\Desktop\\PekkelbakProject\\20.jpg": {
        "latitude": 52.16409863888889,
        "longitude": 4.484417583333333,
        "altitude": 28.0,
        "timestamp": "2018:08:24 17:39:55"
    },
    "C:\\Users\\drift\\Desktop\\PekkelbakProject\\23.jpg": {
        "latitude": 52.164722222222224,
        "longitude": 4.489444444444445,
        "altitude": 0.0,
        "timestamp": "2018:08:24 17:36:32"
    },
    "C:\\Users\\drift\\Desktop\\PekkelbakProject\\24.jpg": {
        "latitude": 52.164638527777775,
        "longitude": 4.489576166666667,
        "altitude": 44.0,
        "timestamp": "2018:08:24 17:32:52"
    },
    "C:\\Users\\drift\\Desktop\\PekkelbakProject\\30.jpg": {
        "latitude": 52.16277777777778,
        "longitude": 4.5,
     

In [9]:
import matplotlib.cm

# create dataframe from extracted data
df = pd.DataFrame(res).T
df['timestamp'] = pd.to_datetime(df.timestamp, format="%Y:%m:%d %H:%M:%S")
df.sort_values('timestamp', inplace=True)

# get colour map
cmap = matplotlib.cm.get_cmap('plasma')
norm = matplotlib.colors.Normalize(vmin=0, vmax=1)
df['color_mapping'] = np.linspace(0, 1, num=df.shape[0])

# Convert colours from RBGA to hex
def rgba_to_hex(rgba: tuple[float, ...]):
    return ('#{:02X}{:02X}{:02X}').format(*rgba[:3])


In [54]:
# calculate bounds SW corner and NE corner, add 3 degrees so that everything fits
sw = (df.latitude.max() + 0.0003, df.longitude.min() - 0.0003)
ne = (df.latitude.min() - 0.0003, df.longitude.max() + 0.0003)

In [55]:
m = folium.Map()

# Add markers
for lat, lon, col, date in zip(df.latitude.values, 
                               df.longitude.values, 
                               df.color_mapping.values, 
                               df.timestamp.values):
    folium.CircleMarker(
        [lat, lon], 
        color=rgba_to_hex(cmap(col, bytes=True)), 
        fill_color=rgba_to_hex(cmap(col, bytes=True)),
        radius=6,
        tooltip = np.datetime_as_string(date, unit='m')
        ).add_to(m)

m.fit_bounds([sw, ne])

In [56]:
m

In [57]:
m.save("the_map.html")